In [567]:
import keras_vggface
import tensorflow as tf
import PIL.Image as Image
import numpy as np
import mtcnn
import scipy.spatial.distance as sp
import matplotlib.pyplot as plt
from mtcnn import MTCNN
from keras_vggface import utils
from keras_vggface.vggface import VGGFace
import os
import pandas as pd
from itertools import chain
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [80]:
cd

/Users/valencialie


In [81]:
cd Desktop/CZ1016_DS2/kdrama/images/Male

/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Male


In [82]:
resnet50_features = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), # use vggface model that is used by resnet50
                                pooling='avg')

In [83]:
def extract_face(filename, required_size=(224, 224)):
    pixels = plt.imread(filename)
    detector = MTCNN()
    results = detector.detect_faces(pixels) # detect face inside a box
    x1, y1, width, height = results[0]['box'] 
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(required_size) # resize to 224,224 because vgg model requires it to be that way 
    face_array = np.asarray(image)
#     plt.imshow(face_array)
#     plt.show()
    pixels = face_array.astype('float32')
    pixels = tf.expand_dims(pixels, axis=0)
    samples = utils.preprocess_input(pixels, version=2) 
    features = resnet50_features.predict(samples)
    return features

In [84]:
path = r"/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Male"
# change the working directory to the path where the images are located
os.chdir(path)

In [85]:
actors = []

In [86]:
# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpeg'):
          # adds only the image files ending with jpeg to the list
            actors.append(file.name)

In [87]:
len(actors)

576

In [88]:
data = {}

for actor in actors:
    try:
        feat = extract_face(actor)
        data[actor] = feat
    except:
        data[actor] = "Blank"

In [89]:
len(data)

576

In [90]:
keys = list(data.keys())

In [91]:
values = list(data.values())

In [92]:
d = { label: value for label, value in zip(keys, values) }

In [93]:
df = pd.DataFrame(d.items(), columns=['Actor', 'Array'])

In [94]:
dist_list = []
dist_list2 = []
index = []
zipped_lists = []
sorted_zipped_lists = []
top10 = []
overall = []

In [95]:
len(d)

576

In [96]:
for j in range(0,576):
    for i in range(0,576):
        if i == j:
            continue
        else:
            dist = sp.cosine(values[i], values[j])
            dist_list.append(dist)
            dist_list2.append(dist)
            index.append(i)
            dist_list.sort()
            top10 = dist_list[:10]
            zipped_lists = zip(dist_list2, index)
            sorted_zipped_lists = sorted(zipped_lists)
            sorted_list1 = [element for _, element in sorted_zipped_lists]
            top10.extend(sorted_list1[:10])
    overall.append(top10)
    dist_list = []
    dist_list2 = []
    index = []
    zipped_lists = []
    sorted_list1 =[]
    top10 = []

In [97]:
df1 = pd.DataFrame(overall,columns=['Least_Array_Diff', '2ndLeast_Array_Diff','3rdLeast_Array_Diff', '4thLeast_Array_Diff', '5thLeast_Array_Diff', '6thLeast_Array_Diff', '7thLeast_Array_Diff','8thLeast_Array_Diff', '9thLeast_Array_Diff', '10thLeast_Array_Diff','1stMatch', '2ndMatch', '3rdMatch', '4thMatch', '5thMatch', '6thMatch', '7thMatch', '8thMatch', '9thMatch', '10thMatch'])

In [98]:
df2 = pd.concat([df, df1], axis = 1)

In [161]:
df2["Actor"] = df2["Actor"].str.replace("_", "")

In [162]:
df2["Actor"] = df2["Actor"].str.replace(".jpeg", "", regex = True)

In [164]:
bestmatch = []

for i in range(0,576):
    bestmatch.append(df2.iloc[df2['1stMatch'][i], 0])
    
secondmatch = []

for i in range(0,576):
    secondmatch.append(df2.iloc[df2['2ndMatch'][i], 0])
    
thirdmatch = []

for i in range(0,576):
    thirdmatch.append(df2.iloc[df2['3rdMatch'][i], 0])
    
fourthmatch = []

for i in range(0,576):
    fourthmatch.append(df2.iloc[df2['4thMatch'][i], 0])

fifthmatch = []

for i in range(0,576):
    fifthmatch.append(df2.iloc[df2['5thMatch'][i], 0])
    
sixthmatch = []

for i in range(0,576):
    sixthmatch.append(df2.iloc[df2['6thMatch'][i], 0])
    
seventhmatch = []

for i in range(0,576):
    seventhmatch.append(df2.iloc[df2['7thMatch'][i], 0])
    
eighthmatch = []

for i in range(0,576):
    eighthmatch.append(df2.iloc[df2['8thMatch'][i], 0])
    
ninthmatch = []

for i in range(0,576):
    ninthmatch.append(df2.iloc[df2['9thMatch'][i], 0])
    
tenthmatch = []

for i in range(0,576):
    tenthmatch.append(df2.iloc[df2['10thMatch'][i], 0])

df3 = pd.DataFrame(bestmatch,columns=['Best_Match_Actor'])
df4 = pd.DataFrame(secondmatch,columns=['Second_Match_Actor'])
df5 = pd.DataFrame(thirdmatch,columns=['Third_Match_Actor'])
df6 = pd.DataFrame(fourthmatch,columns=['Fourth_Match_Actor'])
df7 = pd.DataFrame(fifthmatch,columns=['Fifth_Match_Actor'])
df8 = pd.DataFrame(sixthmatch,columns=['Sixth_Match_Actor'])
df9 = pd.DataFrame(seventhmatch,columns=['Seventh_Match_Actor'])
df10 = pd.DataFrame(eighthmatch,columns=['Eighth_Match_Actor'])
df11 = pd.DataFrame(ninthmatch,columns=['Ninth_Match_Actor'])
df12 = pd.DataFrame(tenthmatch,columns=['Tenth_Match_Actor'])

df13 = pd.concat([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12], axis = 1)

In [101]:
cd

/Users/valencialie


In [102]:
cd Desktop/CZ1016_DS2/kdrama/images/Female

/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Female


In [103]:
actresses = []

In [104]:
path = r"/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Female"
# change the working directory to the path where the images are located
os.chdir(path)

In [105]:
# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpeg'):
          # adds only the image files ending with jpeg to the list
            actresses.append(file.name)

In [106]:
len(actresses)

474

In [107]:
dataF = {}

for actress in actresses:
    try:
        feat = extract_face(actress)
        dataF[actress] = feat
    except:
        dataF[actress] = "Blank"

In [108]:
label = list(dataF.keys())
values = list(dataF.values())

In [175]:
f = { label: value for label, value in zip(label, values) }

In [176]:
dF = pd.DataFrame(f.items(), columns=['Actor', 'Array'])

In [177]:
len(dF)

474

In [112]:
dist_list = []
dist_list2 = []
index = []
zipped_lists = []
sorted_zipped_lists = []
top10 = []
overall = []

In [113]:
for j in range(0,474):
    for i in range(0,474):
        if i == j:
            continue
        else:
            dist = sp.cosine(values[i], values[j])
            dist_list.append(dist)
            dist_list2.append(dist)
            index.append(i)
            dist_list.sort()
            top10 = dist_list[:10]
            zipped_lists = zip(dist_list2, index)
            sorted_zipped_lists = sorted(zipped_lists)
            sorted_list1 = [element for _, element in sorted_zipped_lists]
            top10.extend(sorted_list1[:10])
    overall.append(top10)
    dist_list = []
    dist_list2 = []
    index = []
    zipped_lists = []
    sorted_list1 =[]
    top10 = []

In [115]:
dF1 = pd.DataFrame(overall,columns=['Least_Array_Diff', '2ndLeast_Array_Diff','3rdLeast_Array_Diff', '4thLeast_Array_Diff', '5thLeast_Array_Diff', '6thLeast_Array_Diff', '7thLeast_Array_Diff','8thLeast_Array_Diff', '9thLeast_Array_Diff', '10thLeast_Array_Diff','1stMatch', '2ndMatch', '3rdMatch', '4thMatch', '5thMatch', '6thMatch', '7thMatch', '8thMatch', '9thMatch', '10thMatch'])

In [178]:
dF2 = pd.concat([dF, dF1], axis = 1)

In [180]:
dF2["Actor"] = dF2["Actor"].str.replace("_", "")

In [181]:
dF2["Actor"] = dF2["Actor"].str.replace(".jpeg", "", regex = True)

In [182]:
dF2

,Actor,Array,Least_Array_Diff,2ndLeast_Array_Diff,3rdLeast_Array_Diff,4thLeast_Array_Diff,5thLeast_Array_Diff,6thLeast_Array_Diff,7thLeast_Array_Diff,8thLeast_Array_Diff,...,1stMatch,2ndMatch,3rdMatch,4thMatch,5thMatch,6thMatch,7thMatch,8thMatch,9thMatch,10thMatch
0,JungYooJin,"[[0.0, 0.2543437, 0.0, 4.4913435, 3.635107, 0....",0.275907,0.301436,0.301582,0.313293,0.317468,0.321232,0.321265,0.321366,...,349,377,117,76,468,288,304,58,438,102
1,SeoYiAhn,"[[1.1509761, 0.047182605, 2.0433936, 0.5478225...",0.305233,0.340385,0.346528,0.360276,0.377073,0.377606,0.380573,0.385373,...,148,268,101,91,24,86,84,212,417,388
2,KangYeNa,"[[1.9589509, 0.0, 0.40266016, 0.42564636, 2.02...",0.371118,0.399121,0.402775,0.414845,0.419751,0.429579,0.432363,0.434435,...,67,263,49,162,321,442,62,228,290,260
3,KimHyunSoo,"[[0.0, 0.14416984, 4.598352, 0.061391816, 5.13...",0.305612,0.317215,0.319267,0.340414,0.343880,0.348375,0.352641,0.355292,...,353,329,47,383,135,406,183,207,366,79
4,YooHyeRi,"[[0.47068676, 0.82373685, 1.8157511, 2.4260333...",0.401623,0.409698,0.411395,0.416729,0.426655,0.426857,0.432220,0.441863,...,355,433,354,159,344,356,221,97,438,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,HwangSeungEon,"[[0.0, 0.012250069, 2.016425, 0.021052122, 2.2...",0.226859,0.237822,0.238910,0.275411,0.289282,0.302689,0.305662,0.307152,...,183,15,180,79,93,236,56,361,383,329
470,ChaeSooBin,"[[0.0, 0.27327007, 0.0, 0.0, 3.1540382, 0.0, 0...",0.314409,0.314584,0.316973,0.320586,0.345937,0.351958,0.352943,0.353739,...,430,394,366,465,16,422,18,37,333,179
471,YangJinSung,"[[1.9149007, 0.37245303, 0.0, 1.9541427, 4.512...",0.323316,0.332633,0.360458,0.378607,0.384275,0.384618,0.387496,0.393011,...,292,378,73,352,462,13,25,24,428,101
472,ParkBoYoung,"[[0.0, 0.38084894, 0.0, 0.0, 6.845794, 0.0, 0....",0.246092,0.267348,0.271500,0.276044,0.276334,0.282800,0.297798,0.298549,...,115,373,58,343,394,345,259,139,352,382


In [183]:
bestmatch = []

for i in range(0,474):
    bestmatch.append(dF2.iloc[dF2['1stMatch'][i], 0])
    
secondmatch = []

for i in range(0,474):
    secondmatch.append(dF2.iloc[dF2['2ndMatch'][i], 0])
    
thirdmatch = []

for i in range(0,474):
    thirdmatch.append(dF2.iloc[dF2['3rdMatch'][i], 0])
    
fourthmatch = []

for i in range(0,474):
    fourthmatch.append(dF2.iloc[dF2['4thMatch'][i], 0])

fifthmatch = []

for i in range(0,474):
    fifthmatch.append(dF2.iloc[dF2['5thMatch'][i], 0])
    
sixthmatch = []

for i in range(0,474):
    sixthmatch.append(dF2.iloc[dF2['6thMatch'][i], 0])
    
seventhmatch = []

for i in range(0,474):
    seventhmatch.append(dF2.iloc[dF2['7thMatch'][i], 0])
    
eighthmatch = []

for i in range(0,474):
    eighthmatch.append(dF2.iloc[dF2['8thMatch'][i], 0])
    
ninthmatch = []

for i in range(0,474):
    ninthmatch.append(dF2.iloc[dF2['9thMatch'][i], 0])
    
tenthmatch = []

for i in range(0,474):
    tenthmatch.append(dF2.iloc[dF2['10thMatch'][i], 0])

dF3 = pd.DataFrame(bestmatch,columns=['Best_Match_Actor'])
dF4 = pd.DataFrame(secondmatch,columns=['Second_Match_Actor'])
dF5 = pd.DataFrame(thirdmatch,columns=['Third_Match_Actor'])
dF6 = pd.DataFrame(fourthmatch,columns=['Fourth_Match_Actor'])
dF7 = pd.DataFrame(fifthmatch,columns=['Fifth_Match_Actor'])
dF8 = pd.DataFrame(sixthmatch,columns=['Sixth_Match_Actor'])
dF9 = pd.DataFrame(seventhmatch,columns=['Seventh_Match_Actor'])
dF10 = pd.DataFrame(eighthmatch,columns=['Eighth_Match_Actor'])
dF11 = pd.DataFrame(ninthmatch,columns=['Ninth_Match_Actor'])
dF12 = pd.DataFrame(tenthmatch,columns=['Tenth_Match_Actor'])

dF13 = pd.concat([dF2, dF3, dF4, dF5, dF6, dF7, dF8, dF9, dF10, dF11, dF12], axis = 1)

In [169]:
cd

/Users/valencialie


In [170]:
cd Desktop/CZ1016_DS2/kdrama

/Users/valencialie/Desktop/CZ1016_DS2/kdrama


In [781]:
senti = pd.read_csv("senti.csv")

In [782]:
senti['actor1match'] = np.nan
senti['actor2match'] = np.nan

In [783]:
senti

,Unnamed: 0,pair,actor1,actor2,lineCount,score,actor1match,actor2match
0,0,"('ChaSeungWon', 'LeeSeungGi')",ChaSeungWon,LeeSeungGi,5,0.592857,NaN,NaN
1,1,"('JungHaeIn', 'SonYeJin')",JungHaeIn,SonYeJin,9,0.501087,NaN,NaN
2,2,"('YoonHyunMin', 'GoSungHee')",YoonHyunMin,GoSungHee,5,0.487549,NaN,NaN
3,3,"('SongJiHyo', 'ChoiJinHyuk')",SongJiHyo,ChoiJinHyuk,5,0.478333,NaN,NaN
4,4,"('JungEunJi', 'KimJiSoo')",JungEunJi,KimJiSoo,6,0.466667,NaN,NaN
...,...,...,...,...,...,...,...,...
98,98,"('JoJungSuk', 'ParkBoYoung')",JoJungSuk,ParkBoYoung,8,0.105357,NaN,NaN
99,99,"('YooAhIn', 'KimHeeAe')",YooAhIn,KimHeeAe,5,0.098148,NaN,NaN
100,100,"('ShinHyeSun', 'LeeJongSuk')",ShinHyeSun,LeeJongSuk,5,0.076746,NaN,NaN
101,101,"('KimGoEun', 'ParkHaeJin')",KimGoEun,ParkHaeJin,8,0.025000,NaN,NaN


In [784]:
match1 = []

for j in range (0,103):
    for i in range (0,474):
        if dF13.Best_Match_Actor[i] == senti.actor1[j]:
            match1.append(dF13.Actor[i])
            match1.append(j)

In [785]:
senti["actor1match"]=senti["actor1match"].astype(str)

In [786]:
len(match1)

130

In [787]:
match1

['ShinDongMi',
 3,
 'ParkJooMi',
 6,
 'Eugene',
 6,
 'ParkHyeSoo',
 12,
 'ParkJooMi',
 13,
 'Eugene',
 13,
 'YangJinSung',
 14,
 'SaHyunJin',
 19,
 'HwangSunHee',
 19,
 'ParkJooMi',
 24,
 'Eugene',
 24,
 'NaHaeRyung',
 27,
 'YooInYoung',
 32,
 'YangJinSung',
 35,
 'JeonHyoSung',
 36,
 'YooInNa',
 38,
 'ParkYooNa',
 41,
 'HongSeoYoung',
 47,
 'OhHyunKyung',
 47,
 'BaeDaBin',
 51,
 'YoonSoHee',
 51,
 'JeonSoMin',
 51,
 'LeeRuby',
 51,
 'BaeSuzy',
 51,
 'Yura',
 51,
 'NaHaeRyung',
 53,
 'JungYooJin',
 55,
 'JungHyeSung',
 55,
 'LeeHaNa',
 55,
 'JiWoo',
 55,
 'YooSoYoung',
 55,
 'HanSoHee',
 60,
 'RyuHyunKyung',
 60,
 'MoonSoRi',
 60,
 'BaeDaBin',
 65,
 'YoonSoHee',
 65,
 'JeonSoMin',
 65,
 'LeeRuby',
 65,
 'BaeSuzy',
 65,
 'Yura',
 65,
 'ShinHyeSun',
 66,
 'KimHyunSoo',
 75,
 'HyoMin',
 75,
 'KimHeeJung',
 75,
 'ChoiYeonCheong',
 75,
 'ParkHaSun',
 75,
 'YoonYeJoo',
 75,
 'KimBoMi',
 76,
 'ChunWooHee',
 77,
 'WangJiHye',
 77,
 'JungYuMi',
 77,
 'ParkGyuRi',
 77,
 'JungDaSol',
 77,
 'WonJi

In [788]:
match = []

In [789]:
for i in range(0,130):
    if str(match1[i]).isdigit() == True and senti.iloc[match1[i], 6] == "nan":
        senti.at[match1[i], 'actor1match'] = match1[i-1]
    elif str(match1[i]).isdigit() == True and senti.iloc[match1[i], 6] != "nan":
        match.append(senti.iloc[match1[i], 6])
        match.append(match1[i-1])
        senti.at[match1[i], 'actor1match'] = match
        match = []

In [790]:
senti[senti.actor1match != "nan"]

,Unnamed: 0,pair,actor1,actor2,lineCount,score,actor1match,actor2match
3,3,"('SongJiHyo', 'ChoiJinHyuk')",SongJiHyo,ChoiJinHyuk,5,0.478333,ShinDongMi,NaN
6,6,"('ParkMinYoung', 'SeoKangJoon')",ParkMinYoung,SeoKangJoon,7,0.433478,"[ParkJooMi, Eugene]",NaN
12,12,"('KimSeJeong', 'KimJungHyun')",KimSeJeong,KimJungHyun,6,0.397656,ParkHyeSoo,NaN
13,13,"('ParkMinYoung', 'JiChangWook')",ParkMinYoung,JiChangWook,34,0.397158,"[ParkJooMi, Eugene]",NaN
14,14,"('KimYooJung', 'ParkBoGum')",KimYooJung,ParkBoGum,10,0.393283,YangJinSung,NaN
19,19,"('Uee', 'LeeSeoJin')",Uee,LeeSeoJin,5,0.345421,"[SaHyunJin, HwangSunHee]",NaN
24,24,"('ParkMinYoung', 'LeeMinHo')",ParkMinYoung,LeeMinHo,8,0.331736,"[ParkJooMi, Eugene]",NaN
27,27,"('ParkBoYoung', 'ParkHyungShik')",ParkBoYoung,ParkHyungShik,15,0.319820,NaHaeRyung,NaN
32,32,"('ShinYeEun', 'ParkJinYoung')",ShinYeEun,ParkJinYoung,5,0.304000,YooInYoung,NaN
35,35,"('KimYooJung', 'JiChangWook')",KimYooJung,JiChangWook,9,0.299020,YangJinSung,NaN


In [791]:
match2 = []

for j in range (0,103):
    for i in range (0,474):
        if dF13.Best_Match_Actor[i] == senti.actor2[j]:
            match2.append(dF13.Actor[i])
            match2.append(j)

In [792]:
match2

['BangMinAh',
 1,
 'KimYooJung',
 1,
 'SeoYeJi',
 2,
 'KimSeJeong',
 5,
 'HanJiMin',
 5,
 'SonYeJin',
 11,
 'YooSeoJin',
 11,
 'HwangBoRa',
 11,
 'BangMinAh',
 16,
 'KimYooJung',
 16,
 'JoMinAh',
 18,
 'JungSoMin',
 20,
 'HanBoReum',
 20,
 'JeonHyoSung',
 22,
 'KimDaMi',
 29,
 'MoonGeunYoung',
 29,
 'YeomHyeRan',
 30,
 'JungAeYun',
 30,
 'KimHoJung',
 30,
 'KimHaNeul',
 30,
 'JeonHyeJin',
 30,
 'JeonMiSeon',
 30,
 'LeeMiYoung',
 30,
 'MoonJungHee',
 30,
 'ParkYooNa',
 33,
 'YeomHyeRan',
 34,
 'JungAeYun',
 34,
 'KimHoJung',
 34,
 'KimHaNeul',
 34,
 'JeonHyeJin',
 34,
 'JeonMiSeon',
 34,
 'LeeMiYoung',
 34,
 'MoonJungHee',
 34,
 'ParkYooNa',
 39,
 'NamKiAe',
 40,
 'ParkJiHyun',
 45,
 'LeeElijah',
 45,
 'JeonYeoBin',
 45,
 'ParkJooMi',
 46,
 'Eugene',
 46,
 'KimJaeKyung',
 50,
 'ParkSoonChun',
 50,
 'KimMinJung',
 50,
 'AhnJiHyun',
 56,
 'GoSungHee',
 57,
 'HongSeoYoung',
 58,
 'OhHyunKyung',
 58,
 'KimHyunSoo',
 63,
 'HyoMin',
 63,
 'KimHeeJung',
 63,
 'ChoiYeonCheong',
 63,
 'ParkHaSun

In [793]:
len(match2)

158

In [794]:
senti["actor2match"]=senti["actor2match"].astype(str)

In [795]:
match = []

In [796]:
for i in range(0,158):
    if str(match2[i]).isdigit() == True and senti.iloc[match2[i], 7] == "nan":
        senti.at[match2[i], 'actor2match'] = match2[i-1]
    elif str(match2[i]).isdigit() == True and senti.iloc[match2[i], 7] != "nan":
        match.append(senti.iloc[match2[i], 7])
        match.append(match2[i-1])
        senti.at[match2[i], 'actor2match'] = match
        match = []

In [797]:
senti

,Unnamed: 0,pair,actor1,actor2,lineCount,score,actor1match,actor2match
0,0,"('ChaSeungWon', 'LeeSeungGi')",ChaSeungWon,LeeSeungGi,5,0.592857,nan,nan
1,1,"('JungHaeIn', 'SonYeJin')",JungHaeIn,SonYeJin,9,0.501087,nan,"[BangMinAh, KimYooJung]"
2,2,"('YoonHyunMin', 'GoSungHee')",YoonHyunMin,GoSungHee,5,0.487549,nan,SeoYeJi
3,3,"('SongJiHyo', 'ChoiJinHyuk')",SongJiHyo,ChoiJinHyuk,5,0.478333,ShinDongMi,nan
4,4,"('JungEunJi', 'KimJiSoo')",JungEunJi,KimJiSoo,6,0.466667,nan,nan
...,...,...,...,...,...,...,...,...
98,98,"('JoJungSuk', 'ParkBoYoung')",JoJungSuk,ParkBoYoung,8,0.105357,nan,NaHaeRyung
99,99,"('YooAhIn', 'KimHeeAe')",YooAhIn,KimHeeAe,5,0.098148,nan,nan
100,100,"('ShinHyeSun', 'LeeJongSuk')",ShinHyeSun,LeeJongSuk,5,0.076746,YooInNa,nan
101,101,"('KimGoEun', 'ParkHaeJin')",KimGoEun,ParkHaeJin,8,0.025000,"[[[['JoHyeJoo', 'ChoiARa'], LeeSooJi], KimSeul...",nan


In [798]:
match2 = []

for j in range (0,103):
    for i in range (0,576):
        if df13.Best_Match_Actor[i] == senti.actor2[j]:
            match2.append(df13.Actor[i])
            match2.append(j)

In [799]:
len(match2)

114

In [800]:
match = []

In [801]:
for i in range(0,114):
    if str(match2[i]).isdigit() == True and senti.iloc[match2[i], 7] == "nan":
        senti.at[match2[i], 'actor2match'] = match2[i-1]
    elif str(match2[i]).isdigit() == True and senti.iloc[match2[i], 7] != "nan":
        match.append(senti.iloc[match2[i], 7])
        match.append(match2[i-1])
        senti.at[match2[i], 'actor2match'] = match
        match = []

In [802]:
match2

['LeeHyunJin',
 3,
 'JungKyungHo',
 6,
 'KangHoon',
 6,
 'ParkJiBin',
 8,
 'LeeJooHyung',
 9,
 'ChoiTaeJoon',
 10,
 'ChoiDaniel',
 12,
 'ChoiChanHo',
 12,
 'LeeSooHyuk',
 13,
 'CheonSeungHo',
 14,
 'HongJongHyun',
 14,
 'LeeKangMin',
 14,
 'HongBin',
 14,
 'LeeJaeWook',
 14,
 'LeeHyunWoo',
 14,
 'YoonDooJoon',
 19,
 'JoJungSuk',
 19,
 'YookSungJae',
 21,
 'N',
 21,
 'KimShiHoo',
 24,
 'HwangInYeop',
 25,
 'ChoiTaeJoon',
 26,
 'LeeJinHyuk',
 27,
 'LeeSangYeob',
 32,
 'LeeSooHyuk',
 35,
 'ChoiDaniel',
 38,
 'ChoiChanHo',
 38,
 'KimSungJoo',
 48,
 'KimJungHoon',
 49,
 'ChoiByungChan',
 49,
 'MinHyoGi',
 49,
 'ZuHo',
 49,
 'LeeSeungJun',
 51,
 'BaeInHyuk',
 51,
 'KimJinKwon',
 53,
 'ShimHeeSub',
 53,
 'LeeSooHyuk',
 54,
 'ChoiTaeJoon',
 55,
 'YeoJinGoo',
 60,
 'LeePilMo',
 60,
 'MoonYongSuk',
 60,
 'LeeJoonHyuk',
 66,
 'SonWooHyun',
 67,
 'JangWooYoung',
 67,
 'KwakDongYeon',
 74,
 'ParkJiHoon',
 75,
 'OhMinSuk',
 75,
 'WonHyungHoon',
 75,
 'JooSangWook',
 78,
 'JoHyunJae',
 78,
 'ChoiTaeJ

In [803]:
match1 = []

for j in range (0,103):
    for i in range (0,576):
        if df13.Best_Match_Actor[i] == senti.actor1[j]:
            match1.append(df13.Actor[i])
            match1.append(j)

In [804]:
len(match1)

114

In [805]:
match = []

In [806]:
for i in range(0,114):
    if str(match1[i]).isdigit() == True and senti.iloc[match1[i], 6] == "nan":
        senti.at[match1[i], 'actor1match'] = match1[i-1]
    elif str(match1[i]).isdigit() == True and senti.iloc[match1[i], 6] != "nan":
        match.append(senti.iloc[match1[i], 6])
        match.append(match1[i-1])
        senti.at[match1[i], 'actor1match'] = match
        match = []

In [807]:
senti

,Unnamed: 0,pair,actor1,actor2,lineCount,score,actor1match,actor2match
0,0,"('ChaSeungWon', 'LeeSeungGi')",ChaSeungWon,LeeSeungGi,5,0.592857,"[SungHyuk, BaeSooBin]",nan
1,1,"('JungHaeIn', 'SonYeJin')",JungHaeIn,SonYeJin,9,0.501087,ParkJiBin,"[BangMinAh, KimYooJung]"
2,2,"('YoonHyunMin', 'GoSungHee')",YoonHyunMin,GoSungHee,5,0.487549,"[JangKiYong, Chani]",SeoYeJi
3,3,"('SongJiHyo', 'ChoiJinHyuk')",SongJiHyo,ChoiJinHyuk,5,0.478333,ShinDongMi,LeeHyunJin
4,4,"('JungEunJi', 'KimJiSoo')",JungEunJi,KimJiSoo,6,0.466667,nan,nan
...,...,...,...,...,...,...,...,...
98,98,"('JoJungSuk', 'ParkBoYoung')",JoJungSuk,ParkBoYoung,8,0.105357,nan,NaHaeRyung
99,99,"('YooAhIn', 'KimHeeAe')",YooAhIn,KimHeeAe,5,0.098148,nan,nan
100,100,"('ShinHyeSun', 'LeeJongSuk')",ShinHyeSun,LeeJongSuk,5,0.076746,YooInNa,nan
101,101,"('KimGoEun', 'ParkHaeJin')",KimGoEun,ParkHaeJin,8,0.025000,"[[[['JoHyeJoo', 'ChoiARa'], LeeSooJi], KimSeul...",One
